In [7]:
import json
import os 
import re

In [61]:
PATH_TO_STYLE = "/Users/denyskononenko/Documents/map/data/upload/lun-map.json"
PATH_TO_SPRITES = "/Users/denyskononenko/Documents/map/sprites/"
PATH_TO_SAVE = "/Users/denyskononenko/Documents/map/sprites/"
PATH_TO_METRO = "/Users/denyskononenko/Documents/map/"
METRO_LINES = "metro_lines.geojson"
METRO_ENTRANCE = "metro_entrance.geojson"
METRO_NAMES = "metro_names.geojson"

JSON_FFILENAME = "lun_sprite.json"
RETINA_JSON_FILENAME = "lun_sprite@2x.json"

METRO_LIST = ["metro-kharkiv-blue", "metro-kharkiv-green", "metro-kharkiv-red", "metro-kyiv-blue", "metro-kyiv-green", "metro-kyiv-red"]
PROPERTIES_LIST = ["icon", "railway", "name", "railway", "entrance", "operator", "wheelchair", "transport", "colour", "station", "opening_hours", "ref", "public_transport"]

MOD_METRO_FILENAMES = ["metro_entrance_new.geojson", "metro_names_new.geojson", "metro_lines.geojson"]




In [98]:
def change_down_dash(word):
    numbers = re.search(r"-[0-9][0-9]", word).group(0)
    #print(numbers)
    #print(re.sub(r"_[0-9][0-9]", "-{}{}".format(numbers[0], numbers[1]), re.sub(r"[a-z]{0}-[a-z]{0}", "_", word)))
    return re.sub(r"_[0-9][0-9]", "{}".format(numbers), re.sub(r"[a-z]{0}-[a-z]{0}", "_", word))
     
def fix_metro_json(filename):
    """Change name_ru on name:ru"""
    new_data = {"features": []}
    with open(filename) as data_json:
        data = json.load(data_json)
    
    data_buff = data.copy
    for item in data["features"]:
        temp_obj = {}
        
        temp_obj["type"] = item["type"]
        temp_obj["properties"] = {}
        temp_obj["properties"]["name"] = item["properties"]["name"]
        temp_obj["geometry"] = {}
        temp_obj["id"] = item["id"]
        
        if "name_ru" in item["properties"]:
            #print("name_ru {}".format(item["properties"]["name_ru"]))
            temp_obj["properties"]["name:ru"] = item["properties"]["name_ru"]
        elif "name:ru" in item["properties"]:
            #print("name:ru {}".format(item["properties"]["name:ru"]))
            temp_obj["properties"]["name:ru"] = item["properties"]["name:ru"]
        
        for propert in PROPERTIES_LIST:
            if propert in item["properties"]:
                temp_obj["properties"][propert] = item["properties"][propert]
        
        
        temp_obj["geometry"]["coordinates"] = item["geometry"]["coordinates"]
        temp_obj["geometry"]["type"] = item["geometry"]["type"]
        
        new_data["features"].append(temp_obj)
    
    #print(new_data)
    with open(PATH_TO_METRO + filename.split("/")[-1].split(".")[0] + "_new.geojson", 'w') as outfile:
            json.dump(new_data, outfile, indent=3)
    

def generate_metro_mbtile(filenames_arr):
    filenames = ""
    for filename in filenames_arr:
        filenames += filename + " "
    
    gen_mbt_cmd = "tippecanoe -r1 -o lun_metro.mbtiles %s" % filenames
    os.chdir(PATH_TO_METRO)
    os.system(gen_mbt_cmd)
    print(gen_mbt_cmd)



In [99]:
generate_metro_mbtile(MOD_METRO_FILENAMES)

tippecanoe -r1 -o lun_metro.mbtiles metro_entrance_new.geojson metro_names_new.geojson metro_lines.geojson 


In [58]:
fix_metro_json(PATH_TO_METRO + METRO_NAMES)
fix_metro_json(PATH_TO_METRO + METRO_ENTRANCE)

In [129]:
with open(PATH_TO_SPRITES + JSON_FFILENAME) as testjson:
    data = json.load(testjson)
with open(PATH_TO_SPRITES + RETINA_JSON_FILENAME) as testjson:
    data2 = json.load(testjson)

new_data = data.copy()
new_data2 = data2.copy()

for item in data:
    if item not in METRO_LIST:
        formated = change_down_dash(item)
        if formated != item:
            new_data[formated] = new_data[item]
            del new_data[item]

for item2 in data2:
    if item2 not in METRO_LIST:
        formated2 = change_down_dash(item2)
        if formated2 != item2:
            new_data2[formated2] = new_data2[item2]
            del new_data2[item2]

In [131]:
with open(PATH_TO_SAVE + "lun_sprites_m.json", 'w') as outfile:
            json.dump(new_data, outfile, indent=3)
with open(PATH_TO_SAVE + "lun_sprites_m@2x.json", 'w') as outfile:
            json.dump(new_data2, outfile, indent=3)